<a href="https://colab.research.google.com/github/dave21-py/Image-classification/blob/main/imageclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install duckduckgo-search

In [1]:
# List of folders you want to remove
folders_to_delete = ["__MACOSX", "apple", "orange", "dataset", "old"]

# Loop and remove each folder if it exists
import shutil, os

for folder in folders_to_delete:
    if os.path.exists(folder):
        shutil.rmtree(folder)
        print(f"Deleted folder: {folder}")
    else:
        print(f"Folder not found: {folder}")

Deleted folder: __MACOSX
Deleted folder: apple
Deleted folder: orange
Deleted folder: dataset
Deleted folder: old


In [2]:
!pip install icrawler

In [3]:
from icrawler.builtin import BingImageCrawler

def fetch_images_bing(keyword, folder, max_num=50):
    print(f"Searching for '{keyword}' via Bing …")
    crawler = BingImageCrawler(storage={"root_dir": folder})
    crawler.crawl(keyword=keyword, max_num=max_num)
    print(f"Downloaded {max_num} images into '{folder}'.")

fetch_images_bing("orange fruit", "orange", 50)
print("Download complete!")

Searching for 'orange fruit' via Bing …


ERROR:downloader:Response status code 404, file http://upload.wikimedia.org/wikipedia/commons/thumb/e/e3/Oranges_-_whole-halved-segment.jpg


Downloaded 50 images into 'orange'.
Download complete!


In [4]:
from icrawler.builtin import BingImageCrawler

def fetch_images_bing(keyword, folder, max_num=50):
    print(f"Searching for '{keyword}' via Bing …")
    crawler = BingImageCrawler(storage={"root_dir": folder})
    crawler.crawl(keyword=keyword, max_num=max_num)
    print(f"Downloaded {max_num} images into '{folder}'.")

fetch_images_bing("apple fruit", "apple", 50)
print("Download complete!")

Searching for 'apple fruit' via Bing …
Downloaded 50 images into 'apple'.
Download complete!


In [5]:
!unzip dataset.zip

Archive:  dataset.zip
   creating: dataset/
   creating: dataset/apple/
  inflating: dataset/.DS_Store       
  inflating: __MACOSX/dataset/._.DS_Store  
   creating: dataset/orange/
  inflating: dataset/apple/AdobeStock_840698529_Preview.png  
  inflating: __MACOSX/dataset/apple/._AdobeStock_840698529_Preview.png  
  inflating: dataset/apple/AdobeStock_1209170540_Preview.png  
  inflating: __MACOSX/dataset/apple/._AdobeStock_1209170540_Preview.png  
  inflating: dataset/apple/AdobeStock_59967553_Preview.jpeg  
  inflating: __MACOSX/dataset/apple/._AdobeStock_59967553_Preview.jpeg  
  inflating: dataset/apple/AdobeStock_539609028_Preview.png  
  inflating: __MACOSX/dataset/apple/._AdobeStock_539609028_Preview.png  
  inflating: dataset/apple/AdobeStock_176979696_Preview.jpeg  
  inflating: __MACOSX/dataset/apple/._AdobeStock_176979696_Preview.jpeg  
  inflating: dataset/apple/AdobeStock_1522294100_Preview.png  
  inflating: __MACOSX/dataset/apple/._AdobeStock_1522294100_Preview.png  
 

In [6]:
# Move your personal apple images into the main apple folder
!mv dataset/apple/* apple/

# Move your personal orange images into the main orange folder
!mv dataset/orange/* orange/

print("Your personal images have been merged with the scraped images!")

Your personal images have been merged with the scraped images!


In [7]:
from PIL import Image
import numpy as np
import os

In [8]:
apple_count = len(os.listdir('apple'))
orange_count = len(os.listdir('orange'))
print(f"We have {apple_count} apple images.")
print(f"We have {orange_count} orange images.")

We have 74 apple images.
We have 75 orange images.


In [9]:
first_apple_file = os.listdir('apple')[0]
image_path = os.path.join('apple', first_apple_file)

In [10]:
img = Image.open(image_path)
print("Image details:")
print(f" - Format: {img.format}")
print(f" - Size (Width x Height): {img.size}")
print(f"  - Mode: {img.mode}") # 'RGB' means Red, Green, Blue channels

Image details:
 - Format: JPEG
 - Size (Width x Height): (905, 1000)
  - Mode: RGB


In [11]:
img_as_array = np.array(img)
print(img_as_array[:5,:5,0])

[[255 255 255 255 255]
 [255 255 255 255 255]
 [255 255 255 255 255]
 [255 255 255 255 255]
 [255 255 255 255 255]]


In [12]:
#REsizing

In [13]:
resized_img = img.resize((128,128))
print(f"The new size is: {resized_img.size}")

The new size is: (128, 128)


In [14]:
import torch
import torchvision.transforms as T

In [15]:
#transformer

In [16]:
to_tensor_transformer = T.ToTensor()
img_tensor = to_tensor_transformer(resized_img)

In [18]:
print(f"The type of our new object is: {type(img_tensor)}")
print(f"The shape of the tensor is: {img_tensor.shape}") #x:rgb channels, y:height, z:width

The type of our new object is: <class 'torch.Tensor'>
The shape of the tensor is: torch.Size([3, 128, 128])


In [19]:
#Lets create a pipeline of transformers
#First resize the iamge, then convert it to tensor

In [20]:
transform_pipeline = T.Compose([T.Resize((128,128)), T.ToTensor()])

In [21]:
!mkdir dataset
!mv apple dataset/
!mv orange dataset/

print("Folders organized!")

mkdir: cannot create directory ‘dataset’: File exists
Folders organized!


In [22]:
from torchvision.datasets import ImageFolder

In [23]:
#Create the dataset object
full_dataset = ImageFolder(root='dataset', transform = transform_pipeline)


In [24]:
# Let's check if it worked
print(f"Found {len(full_dataset)} total images.")
print(f"The classes (labels) are: {full_dataset.classes}")

Found 149 total images.
The classes (labels) are: ['apple', 'orange']


In [25]:
#Train test splitting

In [26]:
from torch.utils.data import random_split

In [27]:
#Define the sizes for our split(80% train and 20% test)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size

In [28]:
#Performing the split
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

In [29]:
print(f"Number of images in Training Set: {len(train_dataset)}")
print(f"Number of images in Testing Set: {len(test_dataset)}")

Number of images in Training Set: 119
Number of images in Testing Set: 30


In [30]:
from torch.utils.data import DataLoader

In [31]:
batch_size = 16

In [32]:
#Creating the data loaders
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = batch_size)

In [33]:
print("Data loaders created!")
print(f"The traning loader will serve data in batches of {batch_size}.")

Data loaders created!
The traning loader will serve data in batches of 16.


In [34]:
# Convulutional Neural Network

In [35]:
import torch.nn as nn
import torch.nn.functional as F

In [51]:
class SimpleCNN(nn.Module):
  def __init__(self):
    super().__init__()
    # Define first layer: it takes 3 input channels and produces 16 output feature maps
    self.conv1 = nn.Conv2d(3, 16, kernel_size = 3, padding = 1)
    # Define second layer: it takes. the 16 feature maps from conv1 and produces 32 new ones
    self.conv2 = nn.Conv2d(16, 32, kernel_size = 3, padding = 1)
    # Define a max pooling layer, it downsamples the image, keeping the important information
    self.pool = nn.MaxPool2d(2,2)
    # Define the first fully connected (linear) layer: this starts the classification process
    # The input size(32 * 32 * 32) is calculated from the image size after pooling
    self.fc1 = nn.Linear(32 * 32 * 32, 256)
    # Define the final output layer, it takes 256 inputs and produces 2 outputs (one score for apple and one score for orange)
    self.fc2 = nn.Linear(256, 2)



  #The function defines the forward pass, how data flows through the layers
  def forward(self, x):
    # Data -> conv1 -> activation -> pool
    x = self.pool(F.relu(self.conv1(x)))
    # Data -> conv2 -> activation -> pool
    x = self.pool(F.relu(self.conv2(x)))
    # Now flatten the 2d image into a 1d vector to feed into the linear layers
    x = torch.flatten(x,1)
    # Data -> fc1 -> activation
    x = F.relu(self.fc1(x))
    # Data -> fc2 -> activation
    x = self.fc2(x)
    return x


# Now lets create an instance of our model
model = SimpleCNN()

# Print the model to see its architecture
print(model)

SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=32768, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=2, bias=True)
)


That's the blueprint of the "brain" you just created! Think of it as an assembly line for your image data.

Here's what each part means, step-by-step:

    conv1 (Convolutional Layer 1):

        Job: The first worker on the line. It scans the original image (with its 3 color channels) and looks for 16 very basic patterns, like simple edges or color blobs.

    conv2 (Convolutional Layer 2):

        Job: Takes the 16 simple patterns from conv1 and combines them to look for 32 slightly more complex patterns, like curves or textures.

    pool (Max Pooling Layer):

        Job: A quality control step. It shrinks the pattern maps by half, keeping only the strongest signals. This makes the model more efficient and helps it focus on the most important features.

    fc1 (Fully Connected Layer 1):

        Job: The manager. It takes all the high-level pattern information, which has been flattened into a long list of numbers (in_features=32768), and starts the decision-making process, summarizing it down to 256 key indicators.

    fc2 (Fully Connected Layer 2 - The Final Decision):

        Job: The final boss. It takes the 256 indicators and makes the final call. It outputs 2 numbers. One number represents the model's confidence that the image is an 'apple', and the other is its confidence that it's an 'orange'. The higher number wins.

In [52]:
# LOSS AND OPTIM

In [53]:
import torch.optim as optim

In [54]:
# Define the loss fucntion
criterion = nn.CrossEntropyLoss()

In [55]:
# Define the optmizer
optimizer = optim.Adam(model.parameters(), lr = 0.001)
print("Loss and optim are ready!")

Loss and optim are ready!


In [56]:
# Training Loop

This is the heart of the process. We will create a loop that does the following, over and over again:

    Gets a batch of images and their correct labels from the train_loader.

    Shows the images to the model and gets its predictions.

    Compares the predictions to the correct labels using the criterion (loss function) to see how wrong it was.

    Uses the optimizer to slightly adjust the model's internal connections to make it a little less wrong next time.

We will repeat this process for all the batches in our training set. Going through the entire training set once is called an epoch. We will train for a few epochs.

In [57]:
# No. of epochs
num_epochs = 5

In [60]:
# Main training loop
for epoch in range(num_epochs):
  # variable will track loss for epoch
  running_loss = 0.0
  # now Get batches of data from training loader
  for i, data in enumerate(train_loader, 0):
    # Data object contains a batch of [images, labels]
    inputs, labels = data

    # Step1: Zero the parameter gradients, this is a necessary reset for each batch.
    optimizer.zero_grad()

    # Step2: Forward pass, feed the images(data) to the model to get its predictions(ouputs)
    outputs = model(inputs)

    # Step3: calculate loss, compare the model's outputs to the corrrect labels.
    loss = criterion(outputs, labels)

    # Step4: Backward pass, calculate how much each model param contributed to the error.
    loss.backward()

    # Step5: Optimize, update the models' param to reduce the loss.
    optimizer.step()

    # Add the loss for this batch to the running total
    running_loss += loss.item()
  #Print the avaerage loss for this epoch
  print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

print("Finished training")

Epoch 1, Loss: 1.2194396257400513
Epoch 2, Loss: 0.5175292901694775
Epoch 3, Loss: 0.2860301025211811
Epoch 4, Loss: 0.1631574798375368
Epoch 5, Loss: 0.15033314027823508
Finished training


In [61]:
# Evaluating our model based on ur test data

In [67]:
correct = 0
total = 0

In [68]:
# Set the model to eval mode
model.eval()

SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=32768, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=2, bias=True)
)

In [70]:
# We don't need to calculate gradients for evaluation, so we use torch.no_grad()
# This makes the code run faster and use less memory.
with torch.no_grad():
  # loop over the batches in test loader
  for data in test_loader:
    # Get images, and their true labels from the data batch
    images, labels = data
    # Feed the images to the model
    outputs = model(images)
    # The model outputs a score for each class. We want the class with the highest score.
    # torch.max returns the value and the index of the highest score. We only need the index.
    _, predicted = torch.max(outputs.data, 1)
    # Now, add the no. of images in this batch to our total count
    total += labels.size(0)
    # Add the no. of correct predictions in this batch to our running total
    correct += (predicted == labels).sum().item()


accuracy = 100 * correct / total
print(f"Accuracy of the model on the {len(test_dataset)} test images: {accuracy:.2f}%")


Accuracy of the model on the 30 test images: 75.68%
